Import required modules

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import patient_data
import cnn

Access the folder path for the cancer and the non-cancer images

In [2]:
all_paths = json.loads(open("./paths.json").read())

personal_path = all_paths['personal_path']
non_cancerous_path = personal_path + all_paths['non_cancerous_path']
cancerous_path = personal_path + all_paths['cancerous_path']

Load in all the DICOM files and preprocess/label images

In [3]:
# Using the patient_data data structure, load in all the patient data and save it in a dictionary with the folder name as the key
def load_all_patients(path, add_label = False):
    patients = {}
    folder = os.listdir(path)
    for name in folder:
        patients[name] = patient_data.Patient(os.path.join(path, name))
        if add_label:
            if patients[name].segpath == None:
                print(name, "was not processed correctly")
            else:
                patients[name].label_imgs()
    return patients

# nc_patients = load_all_patients(non_cancerous_path)
c_patients = load_all_patients(cancerous_path, True)


Setting up train/test data

In [4]:
# # not sure if we need this
# # create a list for the merged data
# x = []
# y = []

# create a list for only the cancerous dataset data
x_c = []
y_c = []
# # create a list for only the non-cancerous dataset data
# x_nc = []
# y_nc = []

for patient in c_patients.values():
    for i, img in enumerate(patient.ct.data.values()):
        x_c.append(img)
        y_c.append(patient.labels[i])
        # # not sure if we need this
        # x.append(img)
        # y.append(patient.labels[i])

# for patient in nc_patients.values():
#     for i, img in enumerate(patient.ct.images):
#         x_nc.append(img)
#         y_nc.append(patient.labels[i])
#         # # not sure if we need this
#         # x.append(img)
#         # y.append(patient.labels[i])

# # not sure if we need this
# # Shuffle the merged data
# combined = list(zip(x, y))
# np.random.shuffle(combined)
# x2, y2 = zip(*combined)

def generate_train_test():
    # to ensure equal distribution of non-cancer to cancer data, split the data before merging it
    x_train, x_test, y_train, y_test = train_test_split(x_c, y_c, test_size=0.2, random_state=42)
    # x_train_add, x_test_add, y_train_add, y_test_add = train_test_split(x_nc, y_nc, test_size=0.2, random_state=42)
    # x_train.extend(x_train_add) 
    # x_test.extend(x_test_add) 
    # y_train.extend(y_train_add) 
    # y_test.extend(y_test_add) 

    # Convert lists to arrays
    x_train = np.array(x_train)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    return x_train, x_test, y_train, y_test


Check Class Distribution in Train and Test Sets

In [5]:
# x_train, x_test, y_train, y_test = generate_train_test()

# # Assuming y_train and y_test are your labels for the train and test sets
# train_class_distribution = pd.Series(y_train).value_counts(normalize=True)
# test_class_distribution = pd.Series(y_test).value_counts(normalize=True)

# print(f"Class distribution in training set:")
# print(train_class_distribution)
# print(f"\nClass distribution in testing set:")
# print(test_class_distribution)
# print(f"\nx_train visualization:")
# print(x_train)

Train and test CNN model

In [ ]:
num_tests = 1
cnns = []
for i in range(num_tests):
    x_train, x_test, y_train, y_test = generate_train_test()
    cnns.append(cnn.CNN(x_train, x_test, y_train, y_test))

c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 129s 115s/step - accuracy: 0.7545 - loss: 0.5869 - val_accuracy: 0.8429 - val_loss: 0.3904
Epoch 2/20


c:\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


Cross validation and bootstrapping

In [ ]:
print(cnns[0].test_acc)